## Subarray average of size k

Problem Statement: Given an array, find the average of all contiguous subarrays of size ‘K’ in it.

Brute force is as follows:

In [2]:
def subarray_average(a, k):
    n = len(a)
    ret = []
    
    for i in range(n-k+1):
        ret.append( sum(a[i:i+k]) / k)
    
    return ret

In [3]:
a = [1, 3, 2, 6, -1, 4, 1, 8, 2]
k = 5

In [4]:
subarray_average(a, k)

[2.2, 2.8, 2.4, 3.6, 2.8]

**Time: O(n*k)**

**Space: O(1) (Not including required space for return array)**

With a sliding window, we can do better

In [5]:
def subarray_average_sw(a, k):
    running_sum = sum(a[0:k])
    ret = []
    i, j = 0, k
    
    while j < len(a):
        ret.append(running_sum/k)
        
        running_sum -= a[i]
        running_sum += a[j]
        i += 1
        j += 1
        
    ret.append(running_sum/k)
        
    return ret

In [6]:
subarray_average_sw(a, k)

[2.2, 2.8, 2.4, 3.6, 2.8]

**Time: O(n)**

**Space: O(1)** (Not including required space for return array)

In [7]:
import random
inputs_list = [[random.randint(0, 50) for _ in range(random.randint(1, 20))] for _ in range(50)]

In [8]:
k_list = [random.randint(1, len(a)) for a in inputs_list]

In [9]:
all(subarray_average(a, k)==subarray_average_sw(a, k) for a, k in zip(inputs_list, k_list))

True

## Maximum Sum Subarray of Size K

Problem Statement: Given an array of positive numbers and a positive number ‘k’, find the maximum sum of any contiguous subarray of size ‘k’.

Pretty easy and natural sliding window approach:

In [10]:
import math

def subarray_sum(a, k):
    running_sum = sum(a[0:k])
    max_sum = -math.inf
    i, j = 0, k
    
    while j < len(a):
        max_sum = max(max_sum, running_sum)
        
        running_sum -= a[i]
        running_sum += a[j]
        
        i += 1
        j += 1
        
    max_sum = max(max_sum, running_sum)
    
    return max_sum 

In [60]:
a=[2, 1, 5, 1, 3, 2, 24]
k=3 

In [61]:
subarray_sum(a, k)

29

**Time: O(n)**
    
**Space: O(1)**

## Smallest Subarray with a given sum

Problem Statement: Given an array of positive numbers and a positive number ‘S’, find the length of the smallest contiguous subarray whose sum is greater than or equal to ‘S’. Return 0, if no such subarray exists.

Simple variable length sliding window with ugly edge case handling.

In [28]:
def smallest_subarray(s, arr):
    import math
    
    l, r = 0, 0
    running_sum = arr[0]
    min_len = math.inf
    
    while (r < len(arr) and l <= r):
        if running_sum < s:
            r += 1
            try:
                running_sum += arr[r]
            except:
                pass
        else:
            min_len = min(min_len, r-l+1)
            running_sum -= arr[l]
            l += 1
    return min_len

In [29]:
smallest_subarray(4, [1, 2, 3, 10])

1

**Time: O(n)** - the i, j variable encounter each element at most twice giving O(2n)=O(n) time complexity

**Space: O(1)**

Cleaner solution with no need for explicit edge case handling 

In [55]:
## Cleaner solution
import math

def smallest_subarray_clean(s, arr):
    running_sum, l = 0, 0
    min_len = math.inf
    
    for r in range(0, len(arr)):
        running_sum += arr[r]
        
        while running_sum >= s:
            min_len = min(min_len, r-l+1)
            running_sum -= arr[l]
            l += 1
    
    return min_len if min_len != math.inf else 0

In [56]:
smallest_subarray_clean(4, [1, 2, 3, 10])

1

## Longest Substring with K Distinct Characters

**Problem Statement**: Given a string, find the length of the longest substring in it with no more than K distinct characters.

https://leetcode.com/problems/longest-substring-with-at-most-k-distinct-characters/  

A brute force enumeration can be used here. This would enumerate all $\sum_{j=1}^{n} {n \choose j}$ substrings. By the binomial theorem, this sum is $2^{n}$ and so our time complexity would be exponential. Using a sliding window, we can do better.

In [69]:
from collections import Counter

def longest_substring_with_k_distinct(s, k):
    from collections import Counter
    l = 0
    running_len = 0
    max_len = 0
    seen = Counter()

    for r in range(len(s)):
        seen[s[r]] += 1
        running_len += 1

        while len(seen.keys()) > k: # Loop invariant is violated - fix it
            seen[s[l]] -= 1
            if seen[s[l]] == 0: del seen[s[l]]
            l += 1
            running_len -= 1

        max_len = max(max_len, running_len)

    return max_len

This algorithm is correct because the loop invariant ensures the window under consideration is a valid one (i.e. has k or fewer distinct elements). We consider each such valid window and keep track of the maximum one. If we violate the variant, the while loop executes and restores the invariant by decreasing the window size from the left and only then compares the substring length to the previous maximum substring length. Its clear that the largest len substring will be found because the window is "greedy" in the sense that it will keep increasing the substring until a violation of the invariant occurs. Hence the longest substring will be found.

**Time**: Each element is encoutnered at most twice hence **O(n)** time. The len function might seem like it might be O(n) but actually len for most collection data structures in python is O(1) time.

**Space**: We keep a Counter dictionary to keep track of counts and there can be at most k+1 items in this dictionary hence **O(k)** space

## Fruits into Baskets

**Problem Statement**: Given an array of characters where each character represents a fruit tree, you are given two baskets and your goal is to put maximum number of fruits in each basket. The only restriction is that each basket can have only one type of fruit.

You can start with any tree, but once you have started you can’t skip a tree. You will pick one fruit from each tree until you cannot, i.e., you will stop when you have to pick from a third fruit type.

Write a function to return the maximum number of fruits in both the baskets.

https://leetcode.com/problems/fruit-into-baskets/

Note that this problem is just the previous one with k = 2 but dressed up in a distracting backstory. Be very aware of this type of interview question tactic as it shows up often. You need to ignore the fluff and simplify the problem down to the core problem at hand which is the "Longest Substring with K Distinct Characters" problem. This skill of simplifying down is just as important as the actual algorithmic problem solving part.

In [71]:
def totalFruit(tree):
    return longest_substring_with_k_distinct(s, 2)

Time is same as before, space is O(1) since k=2 now

## No-repeat Substring

Problem Statement: Given a string, find the length of the longest substring which has no repeating characters.

https://leetcode.com/problems/longest-substring-without-repeating-characters/

In [ ]:
def lengthOfLongestSubstring(s):
    l, max_len = 0, 0
    seen = {}
    
    for r, c in enumerate(s):
        if c in seen:
            if seen[c] >= l:
                l = seen[c] + 1
                
        seen[c] = r
        max_len = max(max_len, r-l+1)
        
    return max_len  

The correctness of the algorithm is ensured since, by the loop invariant that we maintain, at the end of the loop, we always have a distinct substring of letters and we are greedily expanding our window thus the maximum distinct substring is guaranteed to be considered. All we need to do is simply keep track of the maximum of all such distinct substrings we see to get the answer.

The only trickiness here compared to the previous 2 problems is that the sliding window "jumps" to shrink. We keep track of all the latest indices we've seen thus far for each character encountered. If we encounter the same character again, we now *potentially* have a repeat. Note that its only potential because if that character is before l (i.e. out of the sliding window) then it does not affect the uniqueness of our sliding window and we *don't* shrink the window. However, if the repeat character *is* in the sliding window (i.e. seen[c] >= l), then we must shrink our window so this character is no longer there (i.e. l = seen[c] + 1). 

**Time**: We only go through the string once hence **O(n)**.

**Space**: The dictionary we have keep track of the unique elements hence **O(k)** where k is the number of unique elements. If s is all unique, the space will be O(n)

## Longest Repeating Character Replacement

**Problem Statement**: https://leetcode.com/problems/longest-repeating-character-replacement/

In [1]:
def characterReplacement(s, k):
    from collections import Counter

    l, max_len, max_repeat = 0, 0, 0
    seen = Counter()

    for r, rc in enumerate(s):
        seen[rc] += 1
        max_repeat = max(max_repeat, seen[rc])

        if (r-l+1-max_repeat) > k: 
            seen[s[l]] -= 1
            l += 1

        max_len = max(max_len, r-l+1)

    return max_len

Note that max_repeat here is *not* keeping track of the current max element of the sliding window. Rather it is a historical max. This is fine because we're only interested in the max substring and this only depends on the historical max (i.e. any substring which does not contain the historical max repeated characters will not be the max substring). So when are window does not contain the historical max, we are moving the window to the right one step at a time. Once we reach a max_repeat greater than the historical max_repeat, we can start increasing our window size again. Overall this solution to the problem is ugly since it doesn't follow intuition and a lot of seemingly incorrect behavior is occurring but in the end, it does indeed keep track of the max substring and thats all we care about.

The best way to think about the sliding window is as follows: We greedily expand until we hit the maximum possible substring. After this we are simply shifting the window to the right one place at a time until the max_repeat character hits a new max and we can then increase our window size once again. The window size only increases or stays the same, and never decreases.

**Time**: **O(n)** since we encounter each element only once and all the ops in the for loop are constant time

**Space**: **O(1)** since we store at most 26 keys in the Counter.

## Longest Subarray with Ones after Replacement

**Problem Statement**: https://leetcode.com/problems/max-consecutive-ones-iii/

In [3]:
def longestOnes(A, K):
    l, max_len = 0, 0
    ones_count = 0

    for r in range(len(A)):
        if A[r] == 1: ones_count += 1

        if r-l+1-ones_count > K: # If the number of zeros is greater than k, shrink window
            if A[l] == 1: ones_count -= 1
            l += 1

        max_len = max(max_len, r-l+1)

    return max_len

This is a simpler version of the above problem. The only possible elements we can replace are the 0's so we only need to keep track of the 1's (as opposed to keeping keeping track of counts of all distinct elements and updating the max_repeat as in the above problem).

The idea is the same as before. We keep greedily expanding the window as long as it's valid (i.e. the number of 0's in the window is less than or equal to k). If it is no longer valid, we shift the window to the right by one spot and try expanding again. It's clear the maximum subarray will be considered in this way and that max_len will keep track of it once it is encounterd.

Time and space same as previous problem

## Permutation in String

Problem Statement: https://leetcode.com/problems/permutation-in-string/

In [6]:
def checkInclusion(s1, s2):
    from collections import Counter
    s1_map, s2_map = Counter(s1), Counter()
    l = 0

    for r, rc in enumerate(s2):
        s2_map[rc] += 1

        if r >= len(s1):
            lc = s2[l]
            s2_map[lc] -= 1
            if s2_map[lc] == 0: del s2_map[lc]
            l += 1

        if s2_map == s1_map: return True
    return False

**Time: O(n)** because both s2_map and s1_map will be 26 keys at most and so the equality check is O(1)

**Space: O(1)** since s1_map and s2_map are O(26) at most.

## String Anagrams

Problem Statment: https://leetcode.com/problems/find-all-anagrams-in-a-string/

In [7]:
def findAnagrams(s, p):
    from collections import Counter

    p_map, s_map = Counter(p), Counter()
    l = 0
    ret = []

    for r, rc in enumerate(s):
        s_map[rc] += 1

        if r >= len(p):
            lc = s[l]
            s_map[lc] -= 1
            if s_map[lc] == 0: del s_map[lc]
            l += 1

        if s_map == p_map:
            ret.append(l)

    return ret

Note that this problem is almost exactly the same as above except now we are simply trying to track ALL such permutations encountered rather than just returning after we encounter the first one. So all we have to change is the eqaulity check: instead of returning true we simply keep track of the start index of the window (l) as that is what the question wants.

Time and space complexity are the same as above (not counting the required extra return array for the space)

## Minimum Window Substring

Problem Statement: https://leetcode.com/problems/minimum-window-substring/

In [12]:
def minWindow(self, s: str, t: str) -> str:
    from collections import Counter

    l, matched, l_ind = 0, 0, 0
    min_len = math.inf
    t_counter = Counter(t)

    for r, rc in enumerate(s):
        if rc in t_counter:
            t_counter[rc] -= 1
            if t_counter[rc] >= 0:
                matched += 1

        # Note that we exit this loop as soon as we lo longer have a 
        # valid window. The search to make it valid begins again at this point
        # with an updated left side of the window.
        while matched == len(t):
            # First we update the min
            if r-l+1 < min_len:
                min_len = r-l+1
                l_ind = l

            # Now we shrink the window. If we already have the minimum
            # window, this loop will not run again. Otherwise, we shrink until
            # we remove the minimal value the makes the window valid.
            lc = s[l]
            l += 1

            if lc in t_counter:
                 # t_counter[c] will either be negative or 0 if we've reached this stage. If it's 0
                 # that means increasing l makes the window invalid and therefore we will be exiting the loop
                if t_counter[lc] == 0:
                    matched -= 1
                t_counter[lc] += 1

    return "" if min_len == math.inf else s[l_ind:l_ind+min_len]

1

The correctness follows because at each expansion of the sliding window, it will try shrinking itself to the smallest possible valid window. Therefore, the smallest such window will undoudtedly be encountered and tracked via min_len and l_ind

**Time**: At most each character is considered twice so **O(n)**

**Space**: The Counter takes **O(k)** space where k is the number of unique letters in t.

## Substring with Concatenation of All Words

Problem Statement: https://leetcode.com/problems/substring-with-concatenation-of-all-words/

In [20]:
def findSubstring(s, words):
    from collections import Counter

    if not words or not s:
        return []

    word_map = Counter(words)
    word_count = len(words)
    word_len = len(words[0])
    ret = []

    for l in range( (len(s)-word_count*word_len)+1):
        word_seen = Counter() # This Counter will start fresh for each start index l under consideration

        for r in range(word_count): 
            nw_ind = l + r*word_len
            nw = s[nw_ind:nw_ind+word_len]

            if nw not in word_map: # We found a word that's not in words, break, this is not a solution
                break

            word_seen[nw] += 1

            if word_seen[nw] > word_map[nw]: # Counted too many of the same word, break, this is not a solution.
                break

            if r+1 == word_count: # All words are in word_seen now and none of them appear more than neccessary.
                ret.append(l)

    return ret

One of the key factors that ensure the correctness of this algorithm is that all words have the same length. Because of this, we can jump word_len by word_len and keep considering each candidate word for each index in the string. If we are able to match all words starting at some position l, then this result gets added to the return array. If even one of the words is not there, or if we count too many of a single word (as seen in the condition word_seen[nw] > word_map[nw]). Overal this is a fairly straightforward algorithm but also not very efficient. However, given the restrictions of the problem, it is an optimal one.

**Time**: **O(len(s) * len(words) * len(words[k])** where k is arbitrary since all words have the same length

**Space**: Two hash maps having at most q entries where **q is the number of unique words in words**. So **O(q)**